### Data types which shpuld be defined in the preliminary dtype file
- bollean: True,False
- binary: 0,1
- duration: 15s, 00:00:15
- categorical: set of strings
- integer: 1,2,3
- string: 'text'
- float: 1.23
- datetime: 1.01.2001 00:00:00
- array: [1,2,3,4]

### case_1

the config file contains 3 binary variables:
- is_Alive: can be 0 if is_Dead is 1
- is_Dead: can be only 1 if is_Alive is 0
- Human: by default each column must have 1 here as everyone in the dataset is Human

how the logic need to process the data
sample_size need to be provided, in put case it would be 10

|id|    Name    |is_Alive|is_Dead|Human|
|--|:-----------|-------:|------:|----:|
|1 |Jake        |1       |0      |1    |
|2 |Nathalie    |1       |0      |1    |
|3 |Owen        |0       |1      |1    |
|4 |Nick        |1       |0      |1    |
|5 |Parla       |0       |1      |1    |
|6 |Peter       |0       |1      |1    |
|7 |Sasha       |1       |0      |1    |
|8 |Madie       |0       |1      |1    |
|9 |Frank       |1       |0      |1    |
|10|Robert      |0       |1      |1    |


Edge cases:
1) Human = 0
2) is_Alive = 0 AND is_Dead = 0
3) is_Dead = 1 AND is_Alive = 1

### Questions to think about
- How to tackle the scalability questions
- How to control the dimensionality when feature number will increase to 1000
- How can I do iterations over binary values (see below)
```
import torch

def gumbel_sigmoid(logits, tau=0.5, eps=1e-9):
    # sample Gumbel noise
    u = torch.rand_like(logits)
    g = -torch.log(-torch.log(u + eps) + eps)
    
    y = (logits + g) / tau
    return torch.sigmoid(y)

# Example
logits = torch.tensor([0.2], requires_grad=True)
tau = 0.3

z_soft = gumbel_sigmoid(logits, tau)
loss = (z_soft - 1)**2
loss.backward()

print(z_soft, logits.grad)
```

### Entities of the dataset
The limitations of the code should not limit the user to generate data outside of the healthcare which will benefit vesatility of the programme and proof the data generation capabilities. Therefore the profound technology analysis need to be done.

Potential entites:
- bank accounts
- clients
- records
- cars
- insurance companies

The technology to be used:
- Copula
- Bayesian Networks for binary variables
- New Datatype for proper configuration
- 

### GPT NOTES
Below is the same content rewritten in strict plain text. No formatting, no emphasis, no bullets beyond simple hyphens. Suitable for specs, docs, or internal standards.

3.1 Mandatory Layers – Deep Specification (plain text)

#### L1 Population Seeder
Purpose: create the initial state of the world. This defines who exists.

*Responsibilities*
- Instantiate base entities such as people, households, accounts, facilities
- Assign immutable attributes
- Ensure population-level consistency with external statistics

*Inputs*
- Aggregated priors such as census or survey data
- Region and year
- Target population size
- Random seed

*Outputs*
- Entity tables with stable, persistent IDs
- No events and no time-dependent outcomes

*Belongs in this layer*
- Sex, date of birth
- Region, LSOA, household membership
- Ethnicity, education if treated as immutable
- Latent baseline variables such as true income or baseline risk

*Must not be in this layer*
- Diseases or diagnoses
- Transactions or visits
- Any time-dependent state

*Design rules*
- Population realism is more important than row-level realism
- IDs must be stable across the full simulation lifecycle
- Latent variables are allowed and encouraged
- Must work without any model training, using priors only

*Typical failure modes*
- Incorrect joint distributions such as age by sex by region
- Unrealistic household structures
- Excessive randomness that destroys structure

⸻

L2 Mechanism Core
Purpose: define how the world actually works at a causal level.

Responsibilities
	•	Encode causal and probabilistic mechanisms
	•	Update latent state over time
	•	React to interventions and policy changes

Inputs
	•	Population entities and their latent state
	•	Current time or time step
	•	Policy parameters
	•	Seeded randomness

Outputs
	•	Updated latent state
	•	Triggered latent events

Core principle
state at t+1 is a function of state at t, causal parents, policy, and noise

Belongs in this layer
	•	True disease status
	•	True income or wealth
	•	Risk exposure and progression
	•	Eligibility or capacity states

Must not be in this layer
	•	Coding systems or administrative labels
	•	Missing data or reporting artifacts
	•	Measurement error

Design rules
	•	Causality must be explicit, not implicit
	•	Interventions must be possible and meaningful
	•	Mechanisms must be inspectable and explainable
	•	All randomness must be controlled and logged

Typical failure modes
	•	Hidden feedback loops
	•	Coupling between unrelated mechanisms
	•	Magic constants without documented provenance

⸻

L3 Policy and Constraint Engine
Purpose: enforce what is allowed to happen in the simulated world.

Responsibilities
	•	Enforce hard constraints that must never be violated
	•	Apply policy regimes and regulatory logic
	•	Modify or block proposed state transitions and events

Inputs
	•	Proposed state transitions or events
	•	Policy configuration and effective dates
	•	Constraint definitions

Outputs
	•	Approved or modified transitions
	•	Rejections with explicit reason codes

Belongs in this layer
	•	Regulatory and ethical rules
	•	Eligibility logic
	•	Scenario-specific policy overrides

Must not be in this layer
	•	Probability estimation
	•	Learning or fitting
	•	Data formatting or export logic

Design rules
	•	Hard and soft constraints must be explicitly separated
	•	Hard constraint violations must fail loudly
	•	Policies must be versioned and time-aware
	•	No silent corrections of invalid states

Typical failure modes
	•	Constraints scattered across code
	•	Silent coercion of invalid values
	•	Hardcoded policy logic

⸻

L4 Event Simulator
Purpose: turn latent reality into time-ordered events.

Responsibilities
	•	Schedule events in time
	•	Maintain causal and temporal ordering
	•	Handle concurrency, delays, and capacity effects

Inputs
	•	Latent state and triggered mechanisms
	•	Time model and calendar rules

Outputs
	•	Deterministic event streams with timestamps

Event properties
	•	Entity identifier
	•	Event type
	•	Start time
	•	End time if applicable
	•	Event attributes still representing latent truth

Belongs in this layer
	•	Visit timing and frequency
	•	Event duration
	•	Seasonality and queues

Must not be in this layer
	•	Coding systems
	•	Missingness
	•	Measurement noise

Design rules
	•	Time must be a first-class concept
	•	Every event must have a cause
	•	Event ordering must be deterministic
	•	Parallel execution must remain reproducible

Typical failure modes
	•	Events occurring before their causes
	•	Non-deterministic scheduling
	•	Implicit randomness in parallel runs

⸻

L5 Observation and Measurement Layer
Purpose: convert latent reality into recorded data.

Responsibilities
	•	Map latent events and states to observed records
	•	Apply coding systems and administrative logic
	•	Inject noise, bias, and missingness

Inputs
	•	Event streams
	•	Observation rules
	•	Coding dictionaries and measurement models

Outputs
	•	Tables that resemble real-world datasets

Belongs in this layer
	•	ICD or billing codes
	•	Measurement error
	•	Missing data mechanisms
	•	Reporting delays and misclassification

Must not be in this layer
	•	Causal logic
	•	Policy enforcement
	•	Population definition

Design rules
	•	Latent truth must remain intact upstream
	•	Observation error must be explicit and configurable
	•	Different systems require different observation layers
	•	Transformations should be traceable and debuggable

Typical failure modes
	•	Mixing true state with observed values
	•	Cleaning or correcting data here
	•	Irreversible transformations

⸻

L6 Exporter
Purpose: materialize shareable datasets with full lineage.

Responsibilities
	•	Map internal tables to target schemas
	•	Enforce schema contracts strictly
	•	Attach manifests and metadata

Inputs
	•	Observed tables
	•	Export schema definitions
	•	Release configuration

Outputs
	•	Files or tables in target formats
	•	Machine-readable manifest
	•	References to validation and privacy reports

Belongs in this layer
	•	Column naming and typing
	•	Units and formatting
	•	Referential integrity enforcement

Must not be in this layer
	•	Domain logic
	•	Simulation behavior
	•	Policy or constraint logic

Design rules
	•	No silent schema coercion
	•	Schema mismatches must fail
	•	Every export must be traceable to a run
	•	Multiple exports per run must be supported

Typical failure modes
	•	Silent type casting
	•	Ambiguous units
	•	Lost lineage information

⸻

Cross-layer invariants
	•	No layer may be skipped
	•	No backward writes across layers
	•	No hidden shared state
	•	No randomness without a seed
	•	Every output must be traceable to config, version, and seed

⸻

Core insight
If any one of these layers is removed or merged improperly, the system stops being production-grade and becomes a demo generator.

$$P(X_1, ..., X_n) = \prod_{i=1}^n P(X_i \mid \text{Parents}(X_i))$$

# Essential Limitations for Now

### Time-series data generation
Time-series data generation is currently limited due to complexity of various sources and rules according to the area of a research. Moreover the complexity of causality of the data from one source to another differessa lot

| Horizon | Engineering Focus        |
| ------- | ------------------------ |
| Short   | Autocorrelation, inertia |
| Medium  | Trend + cycles           |
| Long    | Scenario & assumptions   |


# Seeding

SEED ──▶ PRNG ──▶ uniform numbers ──▶ transformations ──▶ data

1) the hardcoded seeding in the core of the programme 
```python
rng = Random(seed=42)
rng.random()  # always the same result
```

2) yaml structural file seeding
```yaml
seed: 202601
```

3) Each feature seeding
```yaml
features:
  age:
    seed: 1001
  income:
    seed: 1002
```

```
GlobalSeed
 ├─ FeatureSeed
 │   ├─ EntitySeed
 │   │   ├─ TimeSeed
 │   │   └─ NoiseSeed
```

#### Common Engineering Mistakes

- Using one global seed everywhere
- Re-seeding inside loops
- Mixing Python random + NumPy RNGs without control
- Letting seeds depend on execution order
- No seed recorded in metadata

```yaml
dataset_id: v3.1
global_seed: 202601
generator_version: 1.4.2
config_hash: a91f...
```

Essential metadata seeder info:
```yaml
dataset_id: synthetic_v3
global_seed: 202601
config_hash: a91f8c2

```

In [9]:
import torch
import torch.nn.functional as F

def scaled_dot_product_attention(Q, K, V, mask=None):
    d_k = Q.size(-1)
    scores = (Q @ K.transpose(-2, -1)) / (d_k ** 0.5)   # QK^T / sqrt(dk)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    weights = F.softmax(scores, dim=-1)                # softmax(...)
    return weights @ V, weights                        # (...)V

In [ ]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling

# Structure: Age → Disease, Sex → Disease
model = DiscreteBayesianNetwork([
    ("AgeGroup", "Disease"),
    ("Sex", "Disease")
])


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: BayesianNetwork has been deprecated. Please use DiscreteBayesianNetwork instead.

In [3]:
# Example fo the prior:
import numpy as np
from dataclasses import dataclass

rng = np.random.default_rng(seed=42)

# ---------------------------
# PRIORS (WORLD KNOWLEDGE)
# ---------------------------

# Age distribution (piecewise, census-derived)
AGE_BINS = np.array([18, 30, 45, 60, 75, 100])
AGE_PROBS = np.array([0.22, 0.28, 0.25, 0.18, 0.07])  # sums to 1

# Sex prior (almost balanced, small age effect ignored here)
SEX_PROBS = {"M": 0.49, "F": 0.51}

# Smoking prevalence by age bin and sex (expert + survey prior)
SMOKING_RATE = {
    "M": np.array([0.28, 0.30, 0.27, 0.22, 0.15]),
    "F": np.array([0.22, 0.24, 0.21, 0.17, 0.10]),
}

# COPD baseline risk by age bin (non-smokers)
COPD_BASE = np.array([0.002, 0.004, 0.01, 0.03, 0.07])

# Smoking multiplies COPD risk
SMOKING_MULTIPLIER = 4.0


# ---------------------------
# HELPER FUNCTIONS
# ---------------------------

def sample_age():
    bin_idx = rng.choice(len(AGE_PROBS), p=AGE_PROBS)
    low = AGE_BINS[bin_idx]
    high = AGE_BINS[bin_idx + 1]
    return rng.integers(low, high)

def age_bin(age):
    return np.searchsorted(AGE_BINS, age, side="right") - 1


# ---------------------------
# GENERATIVE PROCESS
# ---------------------------

def generate_person():
    # Population seeding
    age = sample_age()
    sex = rng.choice(["M", "F"], p=list(SEX_PROBS.values()))

    # Mechanism: smoking behavior
    bin_idx = age_bin(age)
    smoking_prob = SMOKING_RATE[sex][bin_idx]
    smoker = rng.random() < smoking_prob

    # Mechanism: disease risk
    base_risk = COPD_BASE[bin_idx]
    copd_risk = base_risk * (SMOKING_MULTIPLIER if smoker else 1.0)
    copd = rng.random() < copd_risk

    return {
        "age": age,
        "sex": sex,
        "smoker": smoker,
        "copd_latent": copd,
    }


# ---------------------------
# SYNTHETIC POPULATION
# ---------------------------

population = [generate_person() for _ in range(10_000)]

In [4]:
population

[{'age': np.int64(69),
  'sex': np.str_('F'),
  'smoker': np.False_,
  'copd_latent': np.False_},
 {'age': np.int64(85),
  'sex': np.str_('F'),
  'smoker': np.False_,
  'copd_latent': np.False_},
 {'age': np.int64(37),
  'sex': np.str_('F'),
  'smoker': np.False_,
  'copd_latent': np.False_},
 {'age': np.int64(35),
  'sex': np.str_('M'),
  'smoker': np.False_,
  'copd_latent': np.False_},
 {'age': np.int64(64),
  'sex': np.str_('F'),
  'smoker': np.False_,
  'copd_latent': np.False_},
 {'age': np.int64(69),
  'sex': np.str_('F'),
  'smoker': np.False_,
  'copd_latent': np.False_},
 {'age': np.int64(24),
  'sex': np.str_('F'),
  'smoker': np.False_,
  'copd_latent': np.False_},
 {'age': np.int64(32),
  'sex': np.str_('M'),
  'smoker': np.False_,
  'copd_latent': np.False_},
 {'age': np.int64(26),
  'sex': np.str_('M'),
  'smoker': np.False_,
  'copd_latent': np.False_},
 {'age': np.int64(67),
  'sex': np.str_('F'),
  'smoker': np.False_,
  'copd_latent': np.False_},
 {'age': np.int64(72

In [5]:
# interesting libraries to try:
import stan
import pandas
import pgmpy

In [6]:
schools_code = """
data {
  int<lower=0> J;         // number of schools
  array[J] real y;              // estimated treatment effects
  array[J] real<lower=0> sigma; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

In [7]:
schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

In [8]:
posterior = stan.build(schools_code, data=schools_data, random_seed=1)

RuntimeError: asyncio.run() cannot be called from a running event loop

In [7]:
fit = posterior.sample(num_chains=4, num_samples=1000)

NameError: name 'posterior' is not defined

In [ ]:
eta = fit["eta"]  # array with shape (8, 4000)


---

In [11]:
from config_reader import load_config

cfg = load_config("test_new.yaml")
print(cfg.dataset.n_rows, cfg.dataset.seed)
for col in cfg.columns:
    print(col.name, col.dtype, col.generator.dist, col.generator.params)

10000 42
age int uniform_int {'low': 18, 'high': 90}
sex category categorical {'values': ['F', 'M'], 'probs': [0.52, 0.48]}
has_diabetes bool bernoulli {'p': 0.08}
